# ⚡ abTEM Fast STEM Simulation (AnnularDetector)
This notebook simulates ABF and HAADF images directly using fixed annular detectors.
It is optimized for speed and quick visualization with fewer scan points.

---
## ✅ 1. Environment Setup (Google Colab)

In [ ]:
# Clean install for reproducibility
!pip uninstall -y numpy cupy abtem zarr -q
!pip install numpy==1.26.4
!pip install cupy-cuda117
!pip install abtem
!pip install zarr==2.14.2


## 📂 2. Upload Relaxed Structure (`relaxed_positions_only.cif`)

In [ ]:
from google.colab import files
files.upload()


## 🧱 3. Build Supercell and Align Zone Axis

In [ ]:
from ase.io import read
from ase.build import surface

atoms = read("relaxed_positions_only.cif")
atoms = surface(atoms, indices=(1, 0, 0), layers=4, periodic=True)
atoms.rotate('x', 'z')
atoms = atoms.repeat((2, 2, 1))
atoms.wrap()
atoms.translate((0, -atoms.get_center_of_mass()[1] + atoms.cell[1, 1] / 2, 0))

print("Cell dimensions [Å]:", atoms.cell.lengths())


### 🧭 How to Choose and Align the Zone Axis

abTEM assumes the **beam travels along the z-axis**. So, we rotate the structure to place the desired crystallographic direction along z. Use ASE's `surface()` and `rotate()` to achieve this.

Repeat the unit cell in x and y to enlarge the scanning area. Keep z small (thickness).


## 🔋 4. Create Potential Object

In [ ]:
import abtem
potential = abtem.Potential(atoms, sampling=0.025)
print("Potential grid shape:", potential.shape)


## ⚡ 5. Set Up GPU / CPU for FFT (CuPy or FFTW)

In [ ]:
import cupy
_ = cupy.zeros(1)
try:
    abtem.config.set({"device": "gpu", "fft": "cupy"})
    print("✅ Using GPU with CuPy.")
except Exception as e:
    abtem.config.set({"device": "cpu", "fft": "fftw"})
    print("⚠️ Falling back to CPU. Reason:", e)


## 🔬 6. Define Probe

In [ ]:
probe = abtem.Probe(
    energy=300e3,
    semiangle_cutoff=20,
    Cs=1.3e-3 * 1e10,
    defocus=30
)
probe.grid.match(potential)
print("Probe FWHM [Å]:", probe.profiles().width().compute())


### 🔬 abTEM `Probe` Parameter Guide

- `energy`: Beam energy in eV.
- `semiangle_cutoff`: Max simulated scattering angle in mrad. Detectors must stay within this.
- `Cs`: Spherical aberration (in Å). `1.3 mm = 1.3e-3 * 1e10 Å`.
- `defocus`: Focus offset (positive = overfocus).

Always ensure detectors (next step) use angles < `semiangle_cutoff`.


## 🗺️ 7. Set Scan Grid

In [ ]:
from abtem import GridScan

scan = GridScan(
    start=(0, 0), end=(1, 1),
    sampling=probe.aperture.nyquist_sampling / 3.5,
    fractional=True,
    potential=potential
)
print("Scan shape:", scan.shape)


### 🗺️ GridScan Parameter Guide

Defines how the probe moves across the specimen.  
Start/end are **fractions** of the unit cell.  
Sampling controls resolution. Fewer points = faster but lower-res.

Use `nyquist_sampling / 3.5` to balance detail and speed.


## 🎯 8. Define Annular Detectors and Run Simulation

In [ ]:
from abtem.detectors import AnnularDetector

abf = AnnularDetector(inner=12, outer=19)     # 12–19 mrad
haadf = AnnularDetector(inner=30, outer=32)   # 30–32 mrad (only valid if semiangle_cutoff ≥ 32)

from abtem import stack

result = probe.scan(potential, scan=scan, detectors=[abf, haadf])
result.compute()


### 🎯 How to Choose Annular Detector Ranges

- Must fall within `probe.semiangle_cutoff`, e.g., if cutoff = 20, you cannot use 30–32.
- Typical ranges:
  - ABF: 10–20 mrad
  - HAADF: 30+ mrad

You can print:

```python
print("Max angle:", probe.semiangle_cutoff)
```


## 🖼️ 9. Visualize and Save Output

In [ ]:
fig = result.show(cmap='gray', figsize=(12,5), return_figure=True)
fig.savefig("fast_scan_output.png", dpi=300)
print("Saved image: fast_scan_output.png")


## 🖼️ 10. Post-Processing, Visualization, and Downloads

In this section, we apply Gaussian filtering, save both raw and processed images, and allow for interactive display.

You can adjust the color contrast of ABF using sliders to visually enhance weak contrast features.

This is **critical for interpreting low-angle images**, where subtle differences are informative.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

# Use CPU for post-processing
abtem.config.set({"device": "cpu", "fft": "fftw"})

labels = ["ABF", "HAADF"]
named_stack = result

# Interpolate + filter
interpolated = [m.interpolate(0.05) for m in named_stack]
filtered = [m.gaussian_filter(0.3) for m in interpolated]

# Save and download
for label, raw_img, filt_img in zip(labels, named_stack, filtered):
    raw_array = raw_img.array
    np.save(f"{label.lower()}_raw.npy", raw_array)
    plt.imshow(raw_array, cmap="gray")
    plt.title(f"{label} (raw)")
    plt.colorbar()
    plt.savefig(f"{label.lower()}_raw.png", dpi=300, bbox_inches='tight')
    plt.close()

    files.download(f"{label.lower()}_raw.npy")
    files.download(f"{label.lower()}_raw.png")

    filt_array = filt_img.array
    np.save(f"{label.lower()}_filtered.npy", filt_array)
    plt.imshow(filt_array, cmap="gray")
    plt.title(f"{label} (filtered)")
    plt.colorbar()
    plt.savefig(f"{label.lower()}_filtered.png", dpi=300, bbox_inches='tight')
    plt.close()

    files.download(f"{label.lower()}_filtered.npy")
    files.download(f"{label.lower()}_filtered.png")


## 🔧 11. Interactive ABF Viewer

In [ ]:
import ipywidgets as widgets
from IPython.display import display

abf_measurement = named_stack[0]
abf_array = abf_measurement.array

def show_abf(vmin=0.52, vmax=0.54):
    plt.figure(figsize=(6, 4))
    plt.imshow(abf_array, cmap='gray', vmin=vmin, vmax=vmax)
    plt.title("ABF with adjustable contrast")
    plt.colorbar()
    plt.show()

print("ABF range:", np.min(abf_array), "to", np.max(abf_array))

widgets.interact(show_abf, vmin=(0.50, 0.60, 0.001), vmax=(0.51, 0.62, 0.001));
